# Model Training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
module = os.path.abspath('/home/learner/DLA_project/src/main')
# module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [2]:
manager = DataManager()

In [3]:

manager.load_all()


Loading data from S2
	Path=/media/learner/6663-3462/WESAD/S2/S2.pkl
Loading data from S3
	Path=/media/learner/6663-3462/WESAD/S3/S3.pkl
Loading data from S4
	Path=/media/learner/6663-3462/WESAD/S4/S4.pkl
Loading data from S5
	Path=/media/learner/6663-3462/WESAD/S5/S5.pkl
Loading data from S6
	Path=/media/learner/6663-3462/WESAD/S6/S6.pkl
Loading data from S7
	Path=/media/learner/6663-3462/WESAD/S7/S7.pkl
Loading data from S8
	Path=/media/learner/6663-3462/WESAD/S8/S8.pkl
Loading data from S9
	Path=/media/learner/6663-3462/WESAD/S9/S9.pkl
Loading data from S10
	Path=/media/learner/6663-3462/WESAD/S10/S10.pkl
Loading data from S11
	Path=/media/learner/6663-3462/WESAD/S11/S11.pkl


In [4]:
manager.compute_features();
manager.compute_features_stress();

conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11
conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11


In [5]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)

We have 10  subjects
there are  44461  values for  a_mean
there are  44461  values for  a_std
there are  44461  values for  a_maxx
there are  44461  values for  a_maxy
there are  44461  values for  a_maxz
there are  44461  values for  e_max
there are  44461  values for  e_min
there are  44461  values for  e_mean
there are  44461  values for  e_range
there are  44461  values for  e_std
there are  44461  values for  t_max
there are  44461  values for  t_min
there are  44461  values for  t_mean
there are  44461  values for  t_range
there are  44461  values for  t_std


In [6]:
# Lets go ahead and reshape this data 
X1 = (   manager.FEATURES['a_mean'], manager.FEATURES['a_std'], manager.FEATURES['a_maxx'], manager.FEATURES['a_maxy'], manager.FEATURES['a_maxz'],\
                  manager.FEATURES['e_max'], manager.FEATURES['e_min'], manager.FEATURES['e_mean'], manager.FEATURES['e_range'], manager.FEATURES['e_std'],\
                  manager.FEATURES['t_max'], manager.FEATURES['t_min'], manager.FEATURES['t_mean'], manager.FEATURES['t_range'], manager.FEATURES['t_std'] )
X1 = np.reshape(X1, (len(manager.FEATURES['a_mean'])  ,15))

X2 = (   manager.STRESS_FEATURES['a_mean'], manager.STRESS_FEATURES['a_std'], manager.STRESS_FEATURES['a_maxx'], manager.STRESS_FEATURES['a_maxy'], manager.STRESS_FEATURES['a_maxz'],\
                  manager.STRESS_FEATURES['e_max'], manager.STRESS_FEATURES['e_min'], manager.STRESS_FEATURES['e_mean'], manager.STRESS_FEATURES['e_range'], manager.STRESS_FEATURES['e_std'],\
                  manager.STRESS_FEATURES['t_max'], manager.STRESS_FEATURES['t_min'], manager.STRESS_FEATURES['t_mean'], manager.STRESS_FEATURES['t_range'], manager.STRESS_FEATURES['t_std'] )
X2 = np.reshape(X2, (len(manager.STRESS_FEATURES['a_mean'])  ,15))


In [7]:
# import pip

# try:
#     import keras
# except ImportError:
#     ! sudo pip3 install keras
# try:
#     import h5py
# except ImportError:
#     ! sudo pip3 install h5py
# try:
#     import ibmiotf
# except ImportError:
#     ! sudo pip3 install ibmiotf
# try:
#     import tensorflow
# except ImportError:
#     ! sudo pip3 install tensorflow

In [8]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

Using TensorFlow backend.


It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [9]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [10]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [11]:
print(len(y1))
print(len(y2))

44461
23780


Now we need to concat the data between baseline and stress so that we can split it into training and test sets

In [12]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

(68241, 15)
(68241,)


In [13]:
# Split the data up in train and test sets
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(54592, 15)
(13649, 15)
(54592,)
(13649,)


In [15]:
# Should be

# input_shape=(number of sequences=?, time_steps=None, features=15)

# target=(number of sequences, time_steps, targets)

In [16]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [17]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(54592, 1, 15)
(13649, 1, 15)
(54592,)
(13649,)


In [23]:
print('Building the LSTM network...')


model = Sequential()

model.add(LSTM(32, input_shape=(1, 15), return_sequences=True))

# Note:
# Need to do return_sequences=False for the layer *before* dense
# https://stackoverflow.com/questions/51763983/error-when-checking-target-expected-dense-1-to-have-3-dimensions-but-got-array
model.add(LSTM(32, input_shape=(1, 15), return_sequences=False))
# model.add(LSTM(15, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1, activation='sigmoid'))


Building the LSTM network...


In [19]:
model.summary()

print("inputs: " , model.input_shape)

print("outputs: ", model.output_shape)

print("actual inputs: ")

print("actual outputs: ")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 32)             6144      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              33        
Total params: 6,177
Trainable params: 6,177
Non-trainable params: 0
_________________________________________________________________
inputs:  (None, 1, 15)
outputs:  (None, 1, 1)
actual inputs: 
actual outputs: 


In [24]:
model.compile(loss='binary_crossentropy',

              optimizer='adam',

              metrics=['accuracy'])


In [ ]:
print('Training LSTM...')

batch_size = 1 # I think I want to use batch_size = 1


model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,

                            batch_size=batch_size)

Training LSTM...
Instructions for updating:
Use tf.cast instead.
Train on 54592 samples, validate on 13649 samples
Epoch 1/1
 4663/54592 [=>............................] - ETA: 46:41 - loss: 0.6510 - acc: 0.6455

In [ ]:
print('score:', score)

print('accuracy:', acc)

In [ ]:
# score, acc = model.evaluate(x_test, y_test,

#                             batch_size=batch_size)

print('Test score:', score)

print('Test accuracy:', acc)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Model acc: ", scores[1]*100)


In [ ]:
#manager.train_model()

# manager.test_model()
